# TPS March 2022 - Simple LGBM

In [ ]:
%%capture
!pip install pycaret[full]

import pandas as pd
import numpy as np 
from pycaret.regression import *

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv',index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv',index_col='row_id')

In [ ]:
train['time'] = pd.to_datetime(train.time)
# train on data for Mondays to Thursdays and complete months only
train=train[(train.time.dt.weekday< 4) & (train.time.dt.month > 4)]

## Pre-process

In [ ]:
def pre_process(df):
    
    df['time'] = pd.to_datetime(df['time'])
    df['month']= df.time.dt.month
    df['day']= df.time.dt.dayofyear
    df['am'] = (df.time.dt.hour < 12) & (df.time.dt.hour >6)
    df['wkday'] = df.time.dt.weekday        
    df['time']= (df.time.dt.hour-12)*3+df.time.dt.minute/20
    df['xydirday']= df.x.astype(str)+df.y.astype(str)+df.direction+df.day.astype(str)
    df['xydir'] = df.x.astype(str)+df.y.astype(str)+df.direction
    df['all']= df['xydir']+df.time.astype(str)

In [ ]:
pre_process(train)
pre_process(test)

In [ ]:
# calculate average traffic flow for each time of day 
mapper_avg = train[['all','congestion']].groupby(['all']).median().to_dict()['congestion']

In [ ]:
train['avg']= train['all'].map(mapper_avg)
test['avg']= test['all'].map(mapper_avg)
train= train[train.time >=0]

## Training

In [ ]:
reg = setup(data = train,
            target = 'congestion',
            data_split_shuffle = True, 
            create_clusters = False,
            fold_strategy = 'groupkfold',
            fold_groups = 'wkday',
            use_gpu = True,
            silent = True,
            fold=4,
            ignore_features = ['all','day','xydirday'],
            n_jobs = -1)
model = create_model('lightgbm')

In [ ]:
interpret_model(model)

In [ ]:
final = finalize_model(model)

## Post-process

In [ ]:
test['pred'] = (predict_model(final, data=test)['Label']).round()

# Idea from https://www.kaggle.com/code/ambrosm/tpsmar22-generalizing-the-special-values

sep = train[(train.day >= 246) & (train.time >= 0)]
lower = sep.groupby(['time', 'x', 'y', 'direction']).congestion.quantile(0.15).values
upper = sep.groupby(['time', 'x', 'y', 'direction']).congestion.quantile(0.7).values

test.pred = test.pred.clip(lower, upper)

In [ ]:
# for roadways with low value count replace prediction with nearest value from the training data

for xydir in set(test.xydir):
    
    xydir_counts = train.loc[train.xydir ==xydir,'congestion'].value_counts()
    
    l = xydir_counts[(xydir_counts > 200)] # experimental
    if len(l) > 2: # experimental
        l = list(l.index)
        test.loc[test.xydir ==xydir,'pred']= test.loc[test.xydir ==xydir,'pred'].map(lambda y: min(l, key=lambda x:abs(x-y)))
   

## Submission

In [ ]:
sub = pd.DataFrame(list(zip(test.index,test.pred.values)),columns = ['row_id', 'congestion'])
sub.to_csv('submission.csv', index = False)
sub.head()